# BRATS
The code for the BRATS experiments is organized as follows:
- create_dataset function to create a DVN2 dataset of the BRATS 2018 dataset
- train function to create a DVN2 model according to the specified options that can be trained, saved, used for testing
- perform_analysis function to create results table, ROC plot and figure with qualitative examples as in the paper

## Create the BRATS dataset
First, you have to unzip the training data of BRATS 2018 in a DATASET_DIR directory.
Then, using the ```create_dataset``` function you can convert it to a DVN2 Dataset object.
This can be done using multiple options, e.g. to obtain a "train" or "val" dataset of a specific fold, retain only a certain fraction or even mask the "LGG" or "HGG" cases with zeros.

In [1]:
import numpy as np
from BRATS.dataset import create_dataset
DATASET_DIR ="/usr/local/micapollo01/MIC/DATA/SHARED/STAFF/jberte3/BRATS_Challenge/2018/Raw_data/MICCAI_BraTS_2018_Data_Training"  # to be replaced with correct directory


brats_dataset = create_dataset(DATASET_DIR, data="train", fold_i=0, mask_subset="lgg")
mean_wt_size_1 = np.mean([brats_dataset[case_id]["record_0"]["output"].load().sum() for case_id in brats_dataset])
mean_wt_size_2 = mean_wt_size_1 / 8
print("Mean whole tumor size: {:.0f} (@ 1x1x1) and {:.0f} (@ 2x2x2)".format(mean_wt_size_1, mean_wt_size_2))

Mean whole tumor size: 71554 (@ 1x1x1) and 8944 (@ 2x2x2)


## Create DVN2 model and do training
All experiments of the paper can be run using the ```train``` function and using permutations of the following options:
- $\Phi$ in ["I", "BI", "IC", "BIC"]
- $\epsilon$ in [0, 8789, 12412]
- batch_size in [1, 2, 4, 8]
- fraction in [0.74, 1]

In [1]:
from BRATS.training import train
OUTPUT_DIR = "/usr/local/micapollo01/MIC/DATA/STAFF/jberte3/tmp/datasets/Runs/experiments_of_paper"


train(
    dataset_dir=DATASET_DIR,
    output_dir=OUTPUT_DIR,
    fold_i=0,
    mask_subset=None,
    fraction=1,
    phi="BIC",
    epsilon=1e-7,
    batch_size=2
)

## Perform the analysis
Using the saved model or predictions, we can now create the results using the ```print_table```, ```draw_plots``` and ```draw_figures``` functions.